# <center> USO DE EL SERVICIO BICIMAD POR TEMPORADA  


<center> Marta Simón Arévalo

## 1. INTRODUCCIÓN

La bicicleta es un método de transporte que podemos usar para trasladarnos por Madrid, al igual que lo podemos hacer por medio del transporte público o privado. Suponemos que el uso de un transporte u otro dependerá de la época del año en la que nos encontremos, ya que hay condiciones meteorológicas o referentes al tipo de población que usa este servicio distintas en cada estación. Por tanto, vamos a plantear un estudio para así ver cuáles son las estaciones del año en las que más se utiliza **BiciMad**.

Como en 2014 se empezó a operar el servicio de **BiciMad**, hemos decidido centrarnos en el año 2018 para nuestro análisis. Esto se debe a que ha sido el segundo año más lluvioso en España de la última década, por detrás del año 2010 y creemos que de alguna forma el uso de bicicletas puede verse afectado. Para comparar esto, haremos referencia a los meses de primavera, a otras estaciones y a otro año con aspectos meteorológicos distintos, para así ver los cambios.

En cuanto al estudio por temporadas, cabe destacar que cada una estará representada por una estación del año, es decir, invierno, primavera, verano u otoño. La primera correspondería a los meses de diciembre, enero y febrero; la segunda a marzo, abril y mayo; la tercera a junio, julio y agosto; y, por último, la cuarta a los meses de septiembre, octubre y noviembre. 

Por otra parte, como se trata de un servicio que puede utilizar cualquier persona y puesto que en 2018 la cifra de turistas en la Comunidad de Madrid fue una de la más elevada en los últimos años, dicho grupo puede ser un factor importante a tener en cuenta y que podríamos estudiar más adelante, de forma similar al procedimiento que llevaremos a cabo para tratar las lluvias. De esta forma, estudiaremos los tipos de usuario y las edades de estos a lo largo del año, comparándolo también con otros años. También, se comparará con los meses de otoño para ver la diferencia con la época de reincorporación de trabajadores.

Por último, analizaremos también las estaciones que destacan por mayor enganche y desenganche de bicicletas, así como las que menos uso tienen.

Los datos que vamos a utilizar para nuestro estudio los hemos descargado de **https://opendata.emtmadrid.es/Datos-estaticos/Datos-generales-(1)**. 

A continuación, vamos a exponer las **variables** de nuestra base de datos: 

 **- id_**: identificador del movimiento.
 
 **- user_day_code**: código del usuario para una fecha.
 
 **- idplug_base**: base en la que se engancha la bicicleta.
 
 **- user_type**: tipo de usuario que ha realizado el movimiento. Estos son:
 
    - 0: no se ha podido determinar el tipo de usuario.
    - 1: usuario anual (pase anual).
    - 2: usuario ocasional (perfil turista).
    - 3: trabajador de la empresa.
    - 6: usuario ocasional (no abonado).
    - 7: usuario de BiciMAD GO.
  
 **- idunplug_base**: base de la que se desengancha la bicicleta.
 
 **- travel_time**: tiempo total entre el desenganche y el enganche de la bicicleta (en segundos).
 
 **- idunplug_station**: estación de la que se desengancha la bicicleta.
 
 **- ageRange**: rango de edad del usuario. Siendo:
    
    - 0: no se ha podido determinar el rango de edad del usuario.
    - 1: entre 0 y 16 años.
    - 2: entre 17 y 18 años.
    - 3: entre 19 y 26 años.
    - 4: entre 27 y 40 años.
    - 5: entre 41 y 65 años.
    - 6: 66 años o más.

 **- idplug_station**: estación en la que se engancha la bicicleta.
 
 **- unplug_hourTime**: franja horaria en la que se realiza el desenganche de la bicicleta. Solo se facilita la hora, sin minutos ni segundos, de inicio del movimiento, por cuestiones de anonimato.
 
 **- zip_code**: código postal del usuario que ha realizado el movimiento.
 

Ahora, se importa lo necesario.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
import json
from pprint import pprint

## 2. IMPLEMENTACIÓN EN SPARK

Para poder empezar con el análisis vamos a limpiar la base de datos, dejando las columnas que nos resulten más interesantes para nuestro estudio. 

Nos quedaremos con: **_id**, **ageRange**, **idplug_station**, **idunplug_station**, **travel_time**, **unplug_hourTime**, **user_day_code**, **user_type** y **zip_code**.  

Además, uniremos nuestros datos **por temporadas**, como ya hemos indicado anteriormente.  

In [2]:
def datos(df1,df2,df3):
    d1 = spark.read.json(df1)
    d2 = spark.read.json(df2)
    d3 = spark.read.json(df3)
    df1 = d1.drop(*('_corrupt_record', 'track','idplug_base','idunplug_base')).na.drop("all")
    df2 = d2.drop(*('_corrupt_record', 'track','idplug_base','idunplug_base')).na.drop("all")
    df3 = d3.drop(*('_corrupt_record', 'track','idplug_base','idunplug_base')).na.drop("all")
    return df1.union(df2.union(df3))

In [3]:
invierno = datos('201801_Usage_Bicimad.json','201802_Usage_Bicimad.json','201812_Usage_Bicimad.json')
primavera = datos('201803_Usage_Bicimad.json','201804_Usage_Bicimad.json','201805_Usage_Bicimad.json')
verano = datos('201806_Usage_Bicimad.json','201807_Usage_Bicimad.json','201808_Usage_Bicimad.json')
otoño = datos('201809_Usage_Bicimad.json','201810_Usage_Bicimad.json','201811_Usage_Bicimad.json')

In [4]:
print ('Invierno:')
invierno.printSchema()

print ('Primavera:')
primavera.printSchema()

print ('Verano:')
verano.printSchema()

print ('Otoño:')
otoño.printSchema()

Invierno:
root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- ageRange: long (nullable = true)
 |-- idplug_station: long (nullable = true)
 |-- idunplug_station: long (nullable = true)
 |-- travel_time: long (nullable = true)
 |-- unplug_hourTime: struct (nullable = true)
 |    |-- $date: string (nullable = true)
 |-- user_day_code: string (nullable = true)
 |-- user_type: long (nullable = true)
 |-- zip_code: string (nullable = true)

Primavera:
root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- ageRange: long (nullable = true)
 |-- idplug_station: long (nullable = true)
 |-- idunplug_station: long (nullable = true)
 |-- travel_time: long (nullable = true)
 |-- unplug_hourTime: struct (nullable = true)
 |    |-- $date: string (nullable = true)
 |-- user_day_code: string (nullable = true)
 |-- user_type: long (nullable = true)
 |-- zip_code: string (nullable = true)

Verano:
root
 |-- _id: struct (nullable = true)
 | 

## Comparación entre las estaciones del año

Una vez ya lista nuestra base de datos, podemos empezar calculando **cuántos viajes** se han realizado en cada estación del año: 

In [5]:
print ('En invierno el número de viajes fue: ' + str(invierno.count()))
print ('En primavera el número de viajes fue: ' + str(primavera.count()))
print ('En verano el número de viajes fue: ' + str(verano.count()))
print ('En otoño el número de viajes fue: ' + str(otoño.count()))

En invierno el número de viajes fue: 725472
En primavera el número de viajes fue: 776108
En verano el número de viajes fue: 811329
En otoño el número de viajes fue: 944893


Podemos observar que se han realizado **más viajes durante verano y otoño**, siendo **invierno en la que menos se han hecho** . Por lo que, posteriormente, vamos a estudiar qué tipo de personas son las que utilizan el servicio de BiciMad para hacer un análisis más profundo. 

Además, en relación a los aspectos meteorológicos ya mencionados, estas estaciones son **las menos lluviosas**.

Se crea una **función para ver los datos referentes a cada mes**. Así, se pueden estudiar los que corresponden al verano y los de otoño, para ver si en la primera época crece la cifra de turistas y en la siguiente aumentan los trabajadores que vuelven a la rutina, además de los meses de mayor precipitación.

In [6]:
def meses(df):
    m = spark.read.json(df)
    mes = m.drop(*('_corrupt_record', 'track','idplug_base','idunplug_base')).na.drop("all")
    return mes

#### Relación con el clima

En cuanto al **clima**, como el mes en el que más llovió fue en marzo, vamos a ver si hay alguna diferencia entre los tres meses de primavera.

In [8]:
marzo = meses('201803_Usage_Bicimad.json')
abril = meses('201804_Usage_Bicimad.json')
mayo = meses('201805_Usage_Bicimad.json')

In [9]:
print ('Marzo:')
marzo.groupBy('user_type').sum('travel_time').show()

print ('Abril:')
abril.groupBy('user_type').sum('travel_time').show()

print ('Mayo:')
mayo.groupBy('user_type').sum('travel_time').show()

Marzo:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       172488910|
|        3|        72840210|
|        2|         8357423|
+---------+----------------+

Abril:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       241861803|
|        3|        77857097|
|        2|        14956546|
+---------+----------------+

Mayo:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       214246136|
|        3|        52172611|
|        2|        15236166|
+---------+----------------+



A excepción del grupo 3, los trabajadores de la empresa, **el total de tiempo de trayecto se redujo bastante** para el resto de usuarios. De esta forma, se puede ver que el mes lluvioso ha podido afectar al uso de bicicletas, como habíamos supuesto que sucedería.

In [10]:
print ('En marzo el número de viajes fue: ' + str(marzo.count()))
print ('En abril el número de viajes fue: ' + str(abril.count()))
print ('En mayo el número de viajes fue: ' + str(mayo.count()))

En marzo el número de viajes fue: 215693
En abril el número de viajes fue: 297437
En mayo el número de viajes fue: 262978


Comparando los totales, se ve que, entre los tres meses de primavera, **marzo fue cuando menos viajes se hicieron** y **abril el que más**.

Vamos a probar lo mismo aplicado al **año 2019**, para ver si al hacer mejor tiempo cambió el número de viajes y, por tanto, ver si hay algún tipo de relación entre el clima y el uso de bicicletas.

In [11]:
marzo19 = meses('201903_Usage_Bicimad.json')
abril19 = meses('201904_Usage_Bicimad.json')
mayo19 = meses('201905_Usage_Bicimad.json')

In [12]:
print ('En marzo el número de viajes fue: ' + str(marzo19.count()))
print ('En abril el número de viajes fue: ' + str(abril19.count()))
print ('En mayo el número de viajes fue: ' + str(mayo19.count()))

En marzo el número de viajes fue: 349979
En abril el número de viajes fue: 245527
En mayo el número de viajes fue: 398715


Se observa que **el número de viajes aumentó considerablemente para los meses de marzo y mayo** con respecto del año anterior, aunque se hicieron unos 50000 viajes menos que en 2018.

#### Relación con los turistas y los trabajadores

Se pueden estudiar los que corresponden al **verano** y los de **otoño**, para ver si en la primera época crece la cifra de turistas y en la siguiente aumentan los trabajadores que vuelven a la rutina.

In [13]:
print ('Verano:')
verano.groupBy('user_type').sum('travel_time').show()
print ('Otoño:')
otoño.groupBy('user_type').sum('travel_time').show()

Verano:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       609789864|
|        3|       197096331|
|        2|        27833427|
+---------+----------------+

Otoño:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       725454472|
|        3|       230756392|
|        2|        19501849|
|        0|         2809015|
+---------+----------------+



Se ve que en ambos meses **los usuarios que más han usado este servicio son aquellos que cuentan con un pase anual**, seguidos de los **trabajadores de la empresa** y con un perfil de **turista** en tercer lugar. Además, en septiembre aparecen bastantes segundos en los que no se ha podido determinar el tipo de usuario.

Hagamos una **comparación** también **con el resto de estaciones** por si varían las cifras.

In [14]:
print ('Invierno:')
invierno.groupBy('user_type').sum('travel_time').show()
print ('Primavera:')
primavera.groupBy('user_type').sum('travel_time').show()

Invierno:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       558317570|
|        3|       238866299|
|        2|        20902982|
+---------+----------------+

Primavera:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       628596849|
|        3|       202869918|
|        2|        38550135|
+---------+----------------+



Se observa que en **el resto de meses la estructura es similar**, con menos diferencia entre el tiempo de viaje entre los usuarios anuales y los trabajadores.

A continuación, se lleva a cabo un **análisis más detallado** sobre los tipos de usuarios en estos meses de verano y otoño, además de los del resto del año, con el fin de ver la distribución que siguen.

In [15]:
junio = meses('201806_Usage_Bicimad.json')
julio = meses('201807_Usage_Bicimad.json')
agosto = meses('201808_Usage_Bicimad.json')

In [16]:
print ('Junio:')
junio.groupBy('user_type').sum('travel_time').show()

print ('Julio:')
julio.groupBy('user_type').sum('travel_time').show()

print ('Agosto:')
agosto.groupBy('user_type').sum('travel_time').show()

Junio:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       206493246|
|        3|        57946871|
|        2|         8500781|
+---------+----------------+

Julio:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       236703394|
|        3|        78967515|
|        2|         9856751|
+---------+----------------+

Agosto:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       166593224|
|        3|        60181945|
|        2|         9475895|
+---------+----------------+



In [17]:
print ('En junio el número de viajes fue: ' + str(junio.count()))
print ('En julio el número de viajes fue: ' + str(julio.count()))
print ('En agosto el número de viajes fue: ' + str(agosto.count()))

En junio el número de viajes fue: 271517
En julio el número de viajes fue: 315450
En agosto el número de viajes fue: 224362


Entre estos tres meses, **apenas se produce variación** entre los tipos de usuarios. Lo que sí que cambia es el total de tiempo de viaje, siendo **julio el mes en el que es mayor**, seguido de junio y, finalmente, agosto es cuando menos se usan las bicis dentro del verano, ya que al final mucha gente se desplaza a la playa, por ejemplo, aunque haya turismo.

In [18]:
septiembre = meses('201809_Usage_Bicimad.json')
octubre = meses('201810_Usage_Bicimad.json')
noviembre = meses('201811_Usage_Bicimad.json')

In [19]:
print ('Septiembre:')
septiembre.groupBy('user_type').sum('travel_time').show()

print ('Octubre:')
octubre.groupBy('user_type').sum('travel_time').show()

print ('Noviembre:')
noviembre.groupBy('user_type').sum('travel_time').show()

Septiembre:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       292795101|
|        3|        63163813|
|        2|         9308276|
|        0|         2456711|
+---------+----------------+

Octubre:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        0|          352304|
|        1|       241139072|
|        3|       104178708|
|        2|         6178039|
+---------+----------------+

Noviembre:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       191520299|
|        3|        63413871|
|        2|         4015534|
+---------+----------------+



In [20]:
print ('En septiembre el número de viajes fue: ' + str(septiembre.count()))
print ('En noviembre el número de viajes fue: ' + str(octubre.count()))
print ('En octubre el número de viajes fue: ' + str(noviembre.count()))

En septiembre el número de viajes fue: 368950
En noviembre el número de viajes fue: 322279
En octubre el número de viajes fue: 253664


En este caso, el orden de más a menos segundos, es **como en verano**; con la diferencia de que en septiembre y octubre aparecen usuarios cuyo tipo no se han podido determinar.

Por otra parte, se aprecia que **el perfil turista va disminuyendo su uso** de BiciMad a medida que transcurren los meses de otoño y **el perfil de trabajador de la empresa aumenta considerablemente en el mes de octubre**.

También, se ve un **aumento del total con respecto de los meses de verano**, ya que en septiembre mucha gente se vuelve a incorporar a su día a día de durante el resto del año.

Se estudian entonces las otras dos temporadas restantes.

In [21]:
diciembre = meses('201812_Usage_Bicimad.json')
enero = meses('201801_Usage_Bicimad.json')
febrero = meses('201802_Usage_Bicimad.json')

In [22]:
print ('Diciembre:')
diciembre.groupBy('user_type').sum('travel_time').show()

print ('Enero:')
enero.groupBy('user_type').sum('travel_time').show()

print ('Febrero:')
febrero.groupBy('user_type').sum('travel_time').show()

Diciembre:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       174613178|
|        3|        79666422|
|        2|         5246220|
+---------+----------------+

Enero:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       202810971|
|        3|        91923053|
|        2|         7422020|
+---------+----------------+

Febrero:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       180893421|
|        3|        67276824|
|        2|         8234742|
+---------+----------------+



In [23]:
print ('En diciembre el número de viajes fue: ' + str(diciembre.count()))
print ('En enero el número de viajes fue: ' + str(enero.count()))
print ('En febrero el número de viajes fue: ' + str(febrero.count()))

En diciembre el número de viajes fue: 229667
En enero el número de viajes fue: 263702
En febrero el número de viajes fue: 232103


In [24]:
print ('Marzo:')
marzo.groupBy('user_type').sum('travel_time').show()

print ('Abril:')
abril.groupBy('user_type').sum('travel_time').show()

print ('Mayo:')
mayo.groupBy('user_type').sum('travel_time').show()

Marzo:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       172488910|
|        3|        72840210|
|        2|         8357423|
+---------+----------------+

Abril:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       241861803|
|        3|        77857097|
|        2|        14956546|
+---------+----------------+

Mayo:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       214246136|
|        3|        52172611|
|        2|        15236166|
+---------+----------------+



En los meses correspondientes al **invierno** y a la **primavera**, todos los tipos de usuarios están identificados y el orden de mayor a menor uso de cada tipo de usuario es el **mismo**. En **abril** y en **mayo** **aumentan los usuarios anuales**. Esto puede ser porque son los meses donde hace buen tiempo, pero todavía no es verano.

Un aspecto curioso a destacar es que **solo en septiembre y octubre aparecen usuarios no identificados** según el tipo de pase.

En el año **2019** aumentó el número de turistas que visitaron la ciudad respecto al año anterior. Según los datos proporcionados por **https://www.madrid-destino.com/turismo/estadisticas**, hemos visto que los meses en los que aumentó más el turismo fueron los que forman las estaciones de **Primavera** y **Verano**. Veamos entoces si hay más turistas  que usaron el servicio de **BiciMad** respecto al año 2018.     

In [25]:
verano19 = datos('201906_Usage_Bicimad.json','201907_movements.json','201908_movements.json')
print ('Verano19:')
verano19.groupBy('user_type').sum('travel_time').show()

primavera19 = datos('201903_Usage_Bicimad.json','201904_Usage_Bicimad.json','201905_Usage_Bicimad.json')
print ('Primavera19:')
primavera19.groupBy('user_type').sum('travel_time').show()

Verano19:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       712566472|
|        3|       267948938|
|        2|        25449426|
+---------+----------------+

Primavera19:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       799116284|
|        3|       312331530|
|        2|        34786379|
+---------+----------------+



Aunque esperábamos que el número de usuarios de tipo turista aumentase en estos meses con respecto a estas épocas en el año anterior, no es así. **Aumentaron los tipos 1 y 3, descendiendo el 2.**

Tras conocer esta información, como hemos visto que en **2017** hubo menos turistas que en 2018, especialmente en **Verano** y **Otoño**, veamos entoces si hubo menos turistas que usaron el servicio de **BiciMad** respecto al año 2018. De esta forma, se comprobará si 2019 fue un caso atípico por el tipo de turismo o si, en general, no hay relación entre el turismo y el número de viajes de este tipo de usuario. 

In [26]:
verano17 = datos('201706_Usage_Bicimad.json','201707_Usage_Bicimad.json','201708_Usage_Bicimad.json')
print ('Verano17:')
verano17.groupBy('user_type').sum('travel_time').show()

otoño17 = datos('201709_Usage_Bicimad.json','201710_Usage_Bicimad.json','201711_Usage_Bicimad.json')
print ('Otoño17:')
otoño17.groupBy('user_type').sum('travel_time').show()

Verano17:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       624788722|
|        3|       292576421|
|        2|        21675912|
|        0|        27749379|
+---------+----------------+

Otoño17:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       786757381|
|        3|       320909881|
|        2|        25135549|
|        0|        64013405|
+---------+----------------+



En **Verano de 2017** el número de turistas ha **disminuido** con respecto al año 2018. Pero en **Otoño del 2017** el número de turistas **aumenta**, esto tiene sentido pues como hemos dicho anteriormente (aunque son las estaciones menos lluviosas) en el Otoño de 2018 hubo un aumento notable de las precipitaciones en comparación con 2017.  

También podemos destacar que en **Verano17** aparece el **usuarios no identificados**. Dicho valor no aparece en los datos del 2018, por lo que esto influye también en que disminuya el resto de grupos que aparecen en el análisis. 

## Comparación de las estaciones BiciMad

En esta parte veremos en qué **estaciones** se cogieron las bicicletas en los meses de verano y otoño. De esta forma, se conseguirá ver si siempre se suelen usar las mismas o si, por el contrario, varían dependiendo de qué cliente desenganche la bicicleta.

Decidimos que ya es una estación a destacar si había habido **más de 3500 movimientos** ahí.

In [27]:
print ('Junio:')
pgj = junio.groupBy('idunplug_station').count()
pgj.filter(pgj['count']>3500).show()   

print ('Julio:')
pgjl = julio.groupBy('idunplug_station').count()
pgjl.filter(pgjl['count']>3500).show()

print ('Agosto:')
pga = agosto.groupBy('idunplug_station').count()
pga.filter(pga['count']>3500).show()

Junio:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             149| 3538|
|              57| 4128|
|              43| 4144|
|             175| 4004|
|             163| 4101|
|             135| 4409|
|             129| 3958|
+----------------+-----+

Julio:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             149| 4196|
|              57| 4679|
|              43| 4934|
|              58| 3599|
|               9| 3782|
|             168| 3889|
|             175| 4546|
|             163| 4931|
|             135| 5077|
|             129| 4478|
+----------------+-----+

Agosto:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             175| 3622|
|             135| 3801|
+----------------+-----+



En junio y julio, de donde más bicicletas se recogen son las estaciones siguientes:

- 135: Plaza de San Juan de la Cruz, Ríos Rosas (Chamberí).
- 43: Calle de Lavapiés 36, Embajadores (Centro).
- 57: Calle de Santa Cruz de Marcenado 24, Universidad (Centro).
- 163: Paseo de la Esperanza, Acacias (Arganzuela).
- 175: Calle de Jaime el Conquistador 30, Chopera (Arganzuela).
- 129: Calle de José Abascal 33, Ríos Rosas (Chamberí).
- 149: Paseo de la Habana 42, El Viso (Chamartín).

En agosto, apenas hay estaciones que superen los 3500 movimientos, solo dos. Estas son la 135 y la 175, ubicadas en Chamberí y Arganzuela.

In [28]:
print ('Septiembre:')
pgs = septiembre.groupBy('idunplug_station').count()
pgs.filter(pgs['count']>3500).show()  

print ('Octubre:')
pgo = octubre.groupBy('idunplug_station').count()
pgo.filter(pgo['count']>3500).show()

print ('Noviembre:')
pgn = noviembre.groupBy('idunplug_station').count()
pgn.filter(pgn['count']>3500).show()

Septiembre:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|              19| 4053|
|             149| 4963|
|              57| 5229|
|              43| 4965|
|              31| 3506|
|             161| 3583|
|              58| 3993|
|               9| 3704|
|               1| 4049|
|             168| 4384|
|             132| 3547|
|             160| 3973|
|              83| 3637|
|              49| 3839|
|             162| 3868|
|             175| 5727|
|             163| 5424|
|             135| 6434|
|             129| 5130|
|              90| 3707|
+----------------+-----+
only showing top 20 rows

Octubre:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             149| 4304|
|              57| 4779|
|              43| 4420|
|             168| 4041|
|             160| 3650|
|             162| 3857|
|             175| 4903|
|             163| 5049|
|             135| 4995|
|             129| 4762|
+----------------+-----+

No

En septiembre, destaca el uso de:

- 43: Calle de Lavapiés 36, Embajadores (Centro).
- 149: Paseo de la Habana 42, El Viso (Chamartín).
- 168: Paseo de Santa María de la Cabeza 32, Palos de Moguer (Arganzuela).
- 19: Calle de Prim 4, Justicia (Centro).
- 1: Calle de Alcalá 2 y 6, Sol (Centro).

En octubre, por encima de los 3500 están:

- 163: Paseo de la Esperanza, Acacias (Arganzuela).
- 135: Plaza de San Juan de la Cruz, Ríos Rosas (Chamberí).
- 175: Calle de Jaime el Conquistador 30, Chopera (Arganzuela).
- 57: Calle de Santa Cruz de Marcenado 24, Universidad (Centro).
- 129: Calle de José Abascal 33, Ríos Rosas (Chamberí).

En noviembre:

- 163: Paseo de la Esperanza, Acacias (Arganzuela).
- 175: Calle de Jaime el Conquistador 30, Chopera (Arganzuela).
- 129: Calle de José Abascal 33, Ríos Rosas (Chamberí).
- 43: Calle de Lavapiés 36, Embajadores (Centro).
- 135: Plaza de San Juan de la Cruz, Ríos Rosas (Chamberí).

En general, **las estaciones que destacan durante estas dos temporadas son las mismas**, ubicadas principalmente en los distritos de **Chamberí**, **Centro** y **Arganzuela**. En **septiembre**, destacan algunas estaciones en **Sol** y **Justicia** que, a pesar de estar en el centro de la ciudad, no son tan comunes en el resto de meses.

Se ve **dónde se enganchan** normalmente las bicicletas en los meses anteriores.

In [29]:
print ('Junio:')
pgj = junio.groupBy('idplug_station').count()
pgj.filter(pgj['count']>3500).show()   

print ('Julio:')
pgjl = julio.groupBy('idplug_station').count()
pgjl.filter(pgjl['count']>3500).show()

print ('Agosto:')
pga = agosto.groupBy('idplug_station').count()
pga.filter(pga['count']>3500).show()

Junio:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           149| 3554|
|            57| 4025|
|            43| 4428|
|           175| 4168|
|           163| 4031|
|           135| 4510|
|           129| 3896|
+--------------+-----+

Julio:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           149| 4286|
|            57| 4543|
|            43| 5036|
|             9| 3704|
|           168| 4023|
|           175| 4855|
|           163| 4962|
|           135| 5245|
|           129| 4370|
+--------------+-----+

Agosto:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           175| 3665|
|           135| 3834|
+--------------+-----+



En general, durante el **verano**, las estaciones donde más bicicletas se cogen y dejan suelen ser **las mismas**.

In [30]:
print ('Septiembre:')
pgs = septiembre.groupBy('idplug_station').count()
pgs.filter(pgs['count']>3500).show()  

print ('Octubre:')
pgo = octubre.groupBy('idplug_station').count()
pgo.filter(pgo['count']>3500).show()

print ('Noviembre:')
pgn = noviembre.groupBy('idplug_station').count()
pgn.filter(pgn['count']>3500).show()

Septiembre:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            19| 3953|
|           149| 4985|
|            57| 5141|
|            43| 5125|
|           161| 3677|
|            58| 3829|
|             9| 3639|
|             1| 4008|
|           168| 4480|
|           132| 3546|
|           160| 4043|
|            83| 3651|
|            49| 3922|
|           162| 4053|
|           175| 5825|
|           163| 5459|
|           135| 6655|
|           129| 5071|
|            90| 3664|
|            46| 3768|
+--------------+-----+

Octubre:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           149| 4352|
|            57| 4586|
|            43| 4567|
|           168| 4139|
|           160| 3669|
|           162| 4015|
|           175| 5050|
|           163| 5163|
|           135| 5172|
|           129| 4741|
+--------------+-----+

Noviembre:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            43| 3781|


En **septiembre**, principalmente se usan las estaciones:

- 135: Plaza de San Juan de la Cruz, Ríos Rosas (Chamberí).
- 175: Calle de Jaime el Conquistador 30, Chopera (Arganzuela).
- 163: Paseo de la Esperanza, Acacias (Arganzuela).
- 57: Calle de Santa Cruz de Marcenado 24, Universidad (Centro).
- 43: Calle de Lavapiés 36, Embajadores (Centro).

Exceptuando la de la Calle de Lavapiés, el resto **no son las mismas** que destacan en el desenganche para el mismo mes, pero sí que se encuentran en los **mismos distritos**, sin contar con Chamartín.

En **octubre** y **noviembre**, las estaciones con más desenganches son **las mismas** que las que más enganches tienen.

Ahora veremos en qué **estaciones se desenganchan** menos las bicicletas en los meses de verano y otoño. Así, podremos ver si suelen las mismas o si, por el contrario, varían dependiendo de qué cliente desenganche la bicicleta.

Decidimos que ya es una estación a destacar si había habido **menos de 500 movimientos**.

In [31]:
print ('Junio:')
pgj = junio.groupBy('idunplug_station').count()
pgj.filter(pgj['count']<500).show()   

print ('Julio:')
pgjl = julio.groupBy('idunplug_station').count()
pgjl.filter(pgjl['count']<500).show()

print ('Agosto:')
pga = agosto.groupBy('idunplug_station').count()
pga.filter(pga['count']<500).show()

Junio:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             119|  417|
|              88|  321|
|              28|  449|
|            2008|    1|
+----------------+-----+

Julio:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|              28|  445|
|              88|  354|
|              37|  113|
+----------------+-----+

Agosto:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             144|  346|
|             119|  410|
|              28|  377|
|              88|  256|
|             120|  438|
|              61|  304|
|             173|  369|
|             152|  376|
|            2008|    1|
+----------------+-----+



En **junio, julio y agosto**, notamos que hay dos estaciones comunes donde se recogen las bicicletas:

- 28: Calle de Alcalá (Centro).
- 88: Calle de Velázquez (Salamanca).


En **junio y agosto**, también comparten dos estaciones en común. 

- 119: Calle de Méndez Álvaro (Arganzuela).
- 2008 

La **estación 2008** no aparece en los archivos, sólo tenemos 261 estaciones. A causa de esto, no podemos dar una ubicación exacta de dicha estación.

Este dato puede ser un error de recogida o transcripción, incluso podría tratarse de la estación número 208. Si tomamos la estación como la 208 se trataría de la que está situada en **Paseo de la Castellana** en el distrito de **Chamartín**.

Esta estación es la más destacada pues sólo hay **una recogida** en cada uno de los meses. 

In [32]:
print ('Septiembre:')
pgs = septiembre.groupBy('idunplug_station').count()
pgs.filter(pgs['count']<500).show()  

print ('Octubre:')
pgo = octubre.groupBy('idunplug_station').count()
pgo.filter(pgo['count']<500).show()

print ('Noviembre:')
pgn = noviembre.groupBy('idunplug_station').count()
pgn.filter(pgn['count']<500).show()

Septiembre:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|            2008|    2|
+----------------+-----+

Octubre:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             119|  495|
|            2008|    9|
+----------------+-----+

Noviembre:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|              29|  474|
|             119|  387|
|              28|  294|
|              88|  463|
+----------------+-----+



Observamos que en los meses de **Septiempre** y **Octubre** hay solo una estación en común:

- 2008 --> 208: Paseo de la Castellana (Chamartín).

con una utilización de 2 y 9 viajes respectivamente. Esta es sin duda la estación que menos se usa en **Otoño**. 

En **Noviembre** las estaciones que menos se utilizan son:

- 29: Calle del Marqués de Cubas (Centro).
- 119: Calle de Méndez Álvaro (Arganzuela).
- 28: Calle de Alcalá (Centro).
- 88: Calle de Velázquez (Salamanca). 


En general, **las estaciones que menos se utilizan durante estas dos temporadas**, están en los distritos de **Chamartín**, **Centro**, **Arganzuela** y **Salamanca**. Destacando en especial la que se encuentra en el primer distrito pues en ambas temporadas es la que tiene los valores más bajos. 

Veamos ahora dónde se **enganchan** menos las bicicletas en los meses que hemos estudiado anteriormente.

In [33]:
print ('Junio:')
pgj = junio.groupBy('idplug_station').count()
pgj.filter(pgj['count']<500).show()   

print ('Julio:')
pgjl = julio.groupBy('idplug_station').count()
pgjl.filter(pgjl['count']<500).show()

print ('Agosto:')
pga = agosto.groupBy('idplug_station').count()
pga.filter(pga['count']<500).show()

Junio:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           119|  416|
|            88|  409|
|            28|  423|
|          2008|    1|
+--------------+-----+

Julio:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            88|  444|
|            28|  389|
|            37|   91|
+--------------+-----+

Agosto:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           144|  332|
|           119|  391|
|           107|  468|
|            88|  281|
|            28|  400|
|           120|  434|
|            61|  334|
|           101|  439|
|           173|  377|
|           152|  371|
|          2008|    1|
+--------------+-----+



Al igual que nos sucedió anteriormente, las estaciones donde menos se cogen y se dejan las bicicletas en **Verano** son practicamente las mismas. 

In [34]:
print ('Septiembre:')
pgs = septiembre.groupBy('idplug_station').count()
pgs.filter(pgs['count']<500).show()  

print ('Octubre:')
pgo = octubre.groupBy('idplug_station').count()
pgo.filter(pgo['count']<500).show()

print ('Noviembre:')
pgn = noviembre.groupBy('idplug_station').count()
pgn.filter(pgn['count']<500).show()

Septiembre:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|          2008|    2|
+--------------+-----+

Octubre:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           119|  448|
|          2008|    9|
+--------------+-----+

Noviembre:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            29|  461|
|           119|  384|
|            28|  261|
+--------------+-----+



En **Otoño**, las estaciones donde se enganchan y se desenganchan las bicicletas son las mismas, exceptuando en **noviembre** la estación de la Calle de Velázquez. 

## Comparación por edades

Hacemos un estudio por **edades** para ver si cambia el **público a lo largo del año**.

In [35]:
print ('Junio:')
junio.groupBy('ageRange').count().show()

print ('Julio:')
julio.groupBy('ageRange').count().show()

print ('Agosto:')
agosto.groupBy('ageRange').count().show()

Junio:
+--------+-----+
|ageRange|count|
+--------+-----+
|       0|96998|
|       6| 1254|
|       5|53534|
|       1| 2585|
|       3|18267|
|       2| 3314|
|       4|95565|
+--------+-----+

Julio:
+--------+------+
|ageRange| count|
+--------+------+
|       0|119128|
|       6|  1097|
|       5| 62918|
|       1|  4654|
|       3| 19810|
|       2|  3315|
|       4|104528|
+--------+------+

Agosto:
+--------+-----+
|ageRange|count|
+--------+-----+
|       0|95919|
|       6|  797|
|       5|39557|
|       1| 2539|
|       3|13065|
|       2| 1691|
|       4|70794|
+--------+-----+



In [36]:
print ('Septiembre:')
septiembre.groupBy('ageRange').count().show()

print ('Octubre:')
octubre.groupBy('ageRange').count().show()

print ('Noviembre:')
noviembre.groupBy('ageRange').count().show()

Septiembre:
+--------+------+
|ageRange| count|
+--------+------+
|       0|153639|
|       6|  1416|
|       5| 71918|
|       1|  3907|
|       3| 22550|
|       2|  4051|
|       4|111469|
+--------+------+

Octubre:
+--------+------+
|ageRange| count|
+--------+------+
|       0|135767|
|       6|  1215|
|       5| 64339|
|       1|  3134|
|       3| 19295|
|       2|  3489|
|       4| 95040|
+--------+------+

Noviembre:
+--------+------+
|ageRange| count|
+--------+------+
|       0|104076|
|       6|  1064|
|       5| 51997|
|       1|  3596|
|       3| 15054|
|       2|  2719|
|       4| 75158|
+--------+------+



In [37]:
print ('Diciembre:')
diciembre.groupBy('ageRange').count().show()

print ('Enero:')
enero.groupBy('ageRange').count().show()

print ('Febrero:')
febrero.groupBy('ageRange').count().show()

Diciembre:
+--------+-----+
|ageRange|count|
+--------+-----+
|       0|95214|
|       6| 1006|
|       5|47321|
|       1| 2729|
|       3|13310|
|       2| 2394|
|       4|67693|
+--------+-----+

Enero:
+--------+------+
|ageRange| count|
+--------+------+
|       0| 71343|
|       6|  1398|
|       5| 57835|
|       1|  7242|
|       3| 20207|
|       2|  3586|
|       4|102091|
+--------+------+

Febrero:
+--------+-----+
|ageRange|count|
+--------+-----+
|       0|66920|
|       6| 1163|
|       5|50610|
|       1| 4063|
|       3|17116|
|       2| 3261|
|       4|88970|
+--------+-----+



In [38]:
print ('Marzo:')
marzo.groupBy('ageRange').count().show()

print ('Abril:')
abril.groupBy('ageRange').count().show()

print ('Mayo:')
mayo.groupBy('ageRange').count().show()

Marzo:
+--------+-----+
|ageRange|count|
+--------+-----+
|       0|65628|
|       6|  967|
|       5|45499|
|       1| 3526|
|       3|15388|
|       2| 2942|
|       4|81743|
+--------+-----+

Abril:
+--------+------+
|ageRange| count|
+--------+------+
|       0| 99035|
|       6|  1473|
|       5| 61984|
|       1|  3260|
|       3| 21240|
|       2|  3706|
|       4|106739|
+--------+------+

Mayo:
+--------+-----+
|ageRange|count|
+--------+-----+
|       0|91195|
|       6| 1260|
|       5|53698|
|       1| 2240|
|       3|17464|
|       2| 3214|
|       4|93907|
+--------+-----+



En general, hay una gran cantidad de usuarios cuya edad no se ha podido identificar. 

Principalmente, **destaca el uso por parte de las personas de entre 27 y 40 años, seguido de las de 41 hasta 65**; lo que coincide con la clase trabajadora, que es la que siempre utiliza más este servicio. Aquellos con edades comprendidas **entre 19 y 26 suelen viajar de forma media en bicicleta**, y **los mayores de 66 son los que apenas usan este medio de transporte**, lo cual tiene sentido.

No varía mucho por temporadas, solo **siguen más o menos la misma distribución**, cambiando según los distintos totales.

En el caso de los más **jóvenes**, destaca el uso en el mes de **enero**, siendo mucho más elevado que en mayo o diciembre, meses con las cifras menos elevadas.

También, se puede destacar que el **grupo 2** solo incluye a las personas de 17 y 18 años y aún así **viajan más que el grupo de los ancianos o que los más pequeños**, en ocasiones.

Por último, para los de **mediana edad**, su uso es más o menos **constante** a lo largo del año, dependiendo del clima o de las épocas de trabajo, por ejemplo.

# 3. CONCLUSIÓN

Para terminar, se comenzó agrupando **por estaciones** del año, con tres meses cada una. En **2018**, el número de viajes fue **mayor en otoño**, seguido de verano y con el **menor número en invierno**. La diferencia entre los dos extremos fue de 20000 viajes aproximadamente.

Estudiando ahora el **clima**, como las lluvias se dieron principalmente en el mes de marzo en este año, se compararon los meses de primavera. En **abril fue cuando más uso se dio a BiciMad** y **en marzo**, en relación con este factor meteorológico, **se redujo bastante, con la excepción de los trabajadores de la empresa**.

Para ver si esto tenía sentido, se llevo a cabo la **comparación con 2019**, con mejor clima. Aquí, se observó que **los viajes aumentaron en marzo y mayo**, principalmente.

En general, **los tipos de usuarios son anuales, trabajadores de la empresa y turistas**, con algunos **no identificados en los meses de septiembre y octubre** en el caso de 2018.

Se pensó que en verano hay más turistas y en otoño hay más gente trabajadora que se reincorpora a su rutina, por lo que se podría encontrar algún tipo de relación con el uso de BiciMad. Aunque a lo largo de todo el año **quienes más utilizan el servicio son aquellos que tienen pase anual y los que menos los turistas**, se vio que **en verano había más turistas que en otoño, disminuyendo a lo largo de esta última estación**, y que **en otoño aumentó el número de trabajadores**, destacando principalmente en el mes de octubre. 

Además, dentro del verano el mayor número de viajes se dan en julio y el menor, en agosto.

Después, se compararon los datos con **2019**, año en el que hubo más viajeros en Madrid todavía. **Aumentan todos los usuarios menos los turistas**, lo cual quiere decir que, o el uso de bicicletas depende del tipo de turismo que se lleve a cabo o quizá este año es un dato atípico por cualquier otro motivo que no hemos tenido en cuenta.

Habiendo sucedido esto, se tomó también el año **2017**, con menos turismo que 2018, especialmente en verano y otoño. **En la primera estación hubo menos usuarios de este tipo en 2017, pero más en otoño**. Esto quizá tuvo que ver con las precipitaciones, lo cual tendría sentido.

En cuanto a las estaciones con **más tránsito**, se toman las que tienen **más de 3500 enganches o desenganches**, y para las que **menos** tienen se cogen las que cuentan con una cifra **inferior a 500**.

Para el **desenganche**, en **junio y julio** destacan algunas estaciones de los distritos **Chamberí, Centro y Arganzuela**, aunque en agosto solo hay dos con una cifra elevada (135, 175). En **otoño**, aparecen otras nuevas estaciones como las de las zonas Sol y Justicia y en el resto de meses, se usan mucho algunas de **Arganzuela, Chamberí y Centro**.
**Las que menos se usan en verano se encuentran en el Centro y en Arganzuela, incluyéndose en este grupo algunas de Salamanca en noviembre**. Además, aparece la estación 2008 con muy pocos movimientos que, o es un error de transcripción o hace referencia a la 208, en Paseo de la Castellana, Chamartín.

En el caso del **enganche**, las estaciones son **similares a las de desenganche**, con algunas variaciones en septiembre y una excepción en noviembre.

Por último, se analizó la **edad**, con muchos sin identificar y con un **uso predominante en las personas de mediana edad**, uno **medio en las personas de entre 19 y 26 años**, y **sin viajes apenas en las personas jubiladas**. Esto no varía mucho por temporadas, pero en enero los jóvenes van en bicicleta mucho más que en mayo o diciembre. Por otra parte, el grupo 2 (17-18 años) viaja más en que los ancianos o los pequeños, en total, aunque solo comprendan dos años de edad.